<a href="https://colab.research.google.com/github/memeQueenPaulaDeen/DeepLearnFinalProject/blob/master/_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from pickle import dump
from keras.preprocessing.image import load_img

data_dir = '/content/drive/MyDrive/final_project/data'

train_x = []
train_y = []
test_x = []
test_y = []
for file in listdir(data_dir+'/train'):
  image = load_img(filename, target_size=(224, 224))
  if 'out' in file:
    train_y.append(image)
  else:
    train_x.appened(image)

for file in listdir(data_dir+'/test'):
  image = load_img(filename, target_size=(224, 224))
  if 'out' in file:
    test_y.append(image)
  else:
    test_x.appened(image)

# normalize image pixel values to [0-1] - divide by max value
train_x = train_x.astype('float32')/255.0
test_x = test_x.astype('float32')/255.0

dump(train_x, open(data_dir+'/train_x.pkl', 'wb'))
dump(train_y, open(data_dir+'/train_y.pkl', 'wb'))
dump(test_x, open(data_dir+'/test_x.pkl', 'wb'))
dump(test_y, open(data_dir+'/test_y.pkl', 'wb'))




In [ ]:
base_dir = '/content/drive/MyDrive/final_project'
base_dir_out ='/content/drive/MyDrive/final_project/base' 

In [ ]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Add
from keras.layers import Activation, Dropout, AveragePooling2D, Flatten, concatenate, UpSampling2D
from keras.models import Model, load_model
from keras.optimizers import SGD, Adam, Adagrad, RMSprop, schedules
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.utils import to_categorical, plot_model
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
from pickle import load, dump

def gen_model(input_shape, num_classes):
  #U-Net implementation 
  #https://github.com/zhixuhao/unet/blob/master/model.py
  img_input = Input(img_shape)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(img_input)
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
  drop5 = Dropout(0.5)(conv5)

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
  conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

  # Cost Map extension
  conv11 = Conv2D(num_classes, 3, activation = 'relu', padding = 'same', kerenel_initializer = 'he_normal')(conv10)
  conv11 = Conv2D(num_classes, 3, activation = 'relu', padding = 'same', kerenel_initializer = 'he_normal')(conv10)
  conv11 = Conv2D(1, 3, activation = 'sigmoid', padding = 'same', kerenel_initializer = 'he_normal')(conv10)
  
  model = Model(input = inputs, output = conv10)
  return model


def run_model(base_dir, run_params, model_params):
  ########## Program Variables ##########
  num_epochs, batch_size, optimizer = run_params
  input_shape, num_classes = model_params

  ########## Data Loading ##########
  train_x = load(open(base_dir+'/data/train_x.pkl', 'rb'))
  train_y = load(open(base_dir+'/data/train_y.pkl', 'rb'))
  test_x = load(open(base_dir+'/data/test_x.pkl', 'rb'))
  test_y = load(open(base_dir+'/data/test_y.pkl', 'rb'))

  ########### Generating and Training Model #########
  model = gen_model(input_shape, num_classes)

  model.compile(optimizer = optimizer, loss = 'mse', metrics = ['accuracy'])

  history = model.fit( train_x, train_y, \
            epochs = num_epochs, \
            batch_size = batch_size, \
            validation_data = (test_x, test_y)))

  return history, model

###########################################

num_epochs = 100
batch_size = 128
optimizer = Adam(lr=1e-4)

input_shape = (224, 224, 3)
num_classes = 10

run_params = (num_epochs, batch_size, optimizer)
model_params = (input_shape, num_classes)

history, model = run_model(run_params, model_params)

model.save(base_dir_out+'/model.h5')
dump(history.history, open(base_dir_out+'/history.pkl', 'wb'))


In [ ]:
import matplotlib.pyplot as plt
from keras.models import load_model 
from keras.utils import plot_model
from keras.models import Model, load_model
from pickle import load

def show_history(history):
  plt.figure(1)
  # plot loss
  plt.title('Cross Entropy Loss')
  plt.plot(history['loss'], color='blue', label='train')
  plt.plot(history['val_loss'], color='orange', label='test')
  # plot accuracy
  plt.figure(2)
  plt.title('Classification Accuracy')
  plt.plot(history['accuracy'], color='blue', label='train')
  plt.plot(history['val_accuracy'], color='orange', label='test')

  plt.show()
  print(history['val_accuracy'][-1])
###########################################################

history = load(open(base_dir_out+'history.pkl', 'rb'))
show_history(history)


